# KNN 

In [ ]:
# 1. Exploratory Data Analysis
# 2. Data Preprocessing & Feature Engineering
# 3. Modeling & Prediction
# 4. Model Evaluation
# 5. Hyperparameter Optimization
# 6. Final Model

# 1. Exploratory Data Analysis


In [2]:
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [3]:
df = pd.read_csv("C:/Users/esman/OneDrive/Masaüstü/MachineLearning_datasets/diabetes.csv")

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.shape

(768, 9)

In [6]:
#sayısal değişkenlerin betimsel istatistiskleri
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [7]:
#bağımlı değşkenimin sınıflarının dağılımı
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

# 2. Data Preprocessing & Feature Engineering
Uzaklık temelli yöntemlerde ve gradient descent temelli yöntemlerde değişkenlerin standart olması elde edilecek sonuçların ya daha hızlı ya da daha doğru, daha başarılı olmasını sağlayacaktır. Bu sebeple elimizdeki bağımsız değişkenleri standartlaşma işlemine sokacağız. 

In [8]:
#Bağımlı (y) ve bağımsız değişkenlerini(x) ata.
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

In [9]:
#Bağımsız değişkenleri standartlaştıma işlemi:
#X_scaled = ölçeklendirilmiş X ler.
X_scaled = StandardScaler().fit_transform(X)
X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [ ]:
#X_scaled np.array lerinde sütun isimleri yok bu nedenle sütun isimlerini eklemeliyim
#X_scaledı al. columns sutünlarını = X in ssutunları olarak gir.
X = pd.DataFrame(X_scaled, columns=X.columns)

# 3. Modeling & Prediction


In [10]:
#model kurulumu, modeli eğitmek
knn_model = KNeighborsClassifier().fit(X, y)


In [11]:
#modeli test etmek için random 1 hasta seç.
random_user = X.sample(1, random_state=45)


In [12]:
#eğitilen modelle tahminde bulunma
#predicte girilen argümanlar knn_modele sorulur. ve tahminler alınır.
knn_model.predict(random_user) #tahmin sınıfı 0 olarak geldi. demek ki diyabet değil bu kişi.

#biz burada tek 1 kişi için, 1gözlem için tahminde bulunduk. 
#eğer tüm veri seti için tahminde bulunmak isteseydim "model başarı kriterlerine" gitmem gerekiyor. 

array([0], dtype=int64)

# 4. Model Evaluation
knn modelini kullanarak tüm gözlem birimlerinde tahmşnde bulunacağız.
burada modeli aynı veris eti üzerinde test edeceğiz.
modeli görmediği veride test etmek için 2 yol var; 1.hold-out, 2.cross validation
cross validate birden fazla metriğe göre değerlendirme yapabilmektedir.

In [15]:
#y_pred adında bir değişkende tutuyorum. bunu confusion matrix de kullanacağım. accuracy gibi metrikler bunun üzerinden hesaplancak
y_pred = knn_model.predict(X)

In [16]:
#AUC Değerini hesaplamayı da olasılık değerleri üzerinden yapabiliyorum.
#auc hesaplayabilmek için ROC eğrisine ihtiyacım var. bunun içinde threshold lara ihtiyacım var.
#AUC ve ROC hesaplayabilmek için "1 sınıfına ait olma olasılıklarına"  ihtiyacım var.
y_prob = knn_model.predict_proba(X)[:, 1]


In [17]:
print(classification_report(y, y_pred))


              precision    recall  f1-score   support

           0       0.83      0.88      0.85       500
           1       0.75      0.65      0.70       268

    accuracy                           0.80       768
   macro avg       0.79      0.77      0.78       768
weighted avg       0.80      0.80      0.80       768



In [18]:
roc_auc_score(y, y_prob)


0.8718619402985075

In [20]:
#modelin görmediği veride test etmek istiyorum. croos validation kullanarak. 
#5 katlı çapraz doğrulamayı kullanayım.
#cv_results = cross validation sonuçlarım
#cross_validate (model nesnesi, bağımsız değişken, bağımlı değişken, cv = kaç katlı yapacaksan bunu gir, scoring = cv kullanmak istediğin metrikleri gir.)
cv_results = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])


In [21]:
cv_results

{'fit_time': array([0.0063045 , 0.00506926, 0.00301266, 0.00357795, 0.        ]),
 'score_time': array([0.02803016, 0.01617599, 0.01521945, 0.01208997, 0.01962399]),
 'test_accuracy': array([0.72727273, 0.72727273, 0.7012987 , 0.75816993, 0.70588235]),
 'test_f1': array([0.57142857, 0.58823529, 0.48888889, 0.6407767 , 0.5631068 ]),
 'test_roc_auc': array([0.70592593, 0.71027778, 0.77101852, 0.82537736, 0.73056604])}

In [22]:
cv_results['test_accuracy'].mean()


0.723979288685171

In [23]:
cv_results['test_f1'].mean()

0.5704872499161476

In [24]:
cv_results['test_roc_auc'].mean()

0.7486331236897275

In [27]:
#get_params ile parametreler getirilir.
knn_model.get_params()


{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

# 5. Hyperparameter Optimization
#ayarlamamız gereken dışsal parametreleri programatik bir şekilde en doğru olarak nasıl ayarlayabiliriz?



In [29]:
#kullanıcı tarafından dışsal olarak girilmesi gereken olası hiperparemetreler bir liste içinde tutulut ve modele gösterilir.
#burada en küçük gelen hangisi

In [30]:
knn_model = KNeighborsClassifier()

In [31]:
#parametreleri getirir. komşuluk sayısı 5 olarak ön tanımlı değeri girili.
knn_model.get_params()


{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [32]:
# 2 den 50 ye kadar sayıları oluşturdum,ve bunlar tek tek denenecek
knn_params = {"n_neighbors": range(2, 50)}


In [33]:
#GridSearchCV yöntemi = ızgarada cv ile ara.
knn_gs_best = GridSearchCV(knn_model,                 #modeli gir
                           knn_params,                #hangi parametre seti  
                           cv=5,                      #kaç katlı cv yapacaksın
                           n_jobs=-1,                 # -1 ile tanımlanınca işlemcilerini tam performansda kullanır.
                           verbose=1).fit(X, y)       #verbose ile rapor edilir.
knn_gs_best.best_params_

Fitting 5 folds for each of 48 candidates, totalling 240 fits


{'n_neighbors': 14}

In [34]:
#komşuluk sayısı 14 geldi. ön tanımlı değerim 5 idi.
knn_gs_best.best_params_

{'n_neighbors': 14}

# FİNAL MODEL
HİPERMETRE OPTİMİZASYONU ile komşuluk sayım 17 olmuştu. ben biraz önce modelimi kurdum ama bu model komşuluk sayısı 5 e göre kuruldu. şimdi 17 ile daha başarılı sonuçlar vereceğini düşünüyorum. 

In [36]:
knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X, y)

In [37]:
#test hatası
cv_results = cross_validate(knn_final,
                            X,
                            y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])

In [38]:
cv_results['test_accuracy'].mean()


0.7578558696205755

In [39]:
cv_results['test_f1'].mean()


0.5870824671121806

In [40]:
cv_results['test_roc_auc'].mean()


0.7957707896575821

In [41]:
random_user = X.sample(1)


In [ ]:
knn_final.predict(random_user)